### 1st CNN predictions

In [ ]:
probability_model = tensorflow.keras.Sequential([wholemodel_cnn,
                                         tensorflow.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)
plot_prediction_images(predictions)

In [ ]:
predictions

### 2nd CNN predictions

In [ ]:
probability_model2 = tensorflow.keras.Sequential([wholemodel_cnn2,
                                         tensorflow.keras.layers.Softmax()])
predictions2 = probability_model2.predict(test_images)
plot_prediction_images(predictions2)

### VGG16 predictions

In [ ]:
probability_model = tensorflow.keras.Sequential([wholemodel,
                                         tensorflow.keras.layers.Softmax()])
predictions_vgg = probability_model.predict(test_images)
plot_prediction_images(predictions_vgg)

# Test avec images de base

In [ ]:
train_image,test_image,train_label, test_label = train_test_split(data_original.loc[:,['Image','Grayscale']],data.loc[:,['Race','RaceId']],stratify=data['RaceId'],test_size=0.1)

In [ ]:
y_counts = pd.DataFrame({
    'data': np.array(['train'] * num_classes + ['test'] * num_classes),
    'class': np.tile(np.arange(num_classes), 2),
    'prop': np.hstack([np.bincount(train_label['RaceId']) / train_label.shape[0],
                         np.bincount(test_label['RaceId']) / test_label.shape[0]])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x='class', y='prop', hue='data', data=y_counts, ax=ax)

In [ ]:
plt.figure(figsize=(10,10))
k=0
for i in train_image.index[:25]:
    plt.subplot(5,5,k+1)
    k+=1
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_image['Image'][i])
    plt.xlabel(train_label['Race'][i])
plt.show()

We normalize the data:

In [ ]:
train_images = train_image['Image']
test_images = test_image['Image']

train_labels=train_label['RaceId']
test_labels=test_label['RaceId']


In [ ]:
plt.figure()
plt.imshow(data['Grayscale'][0])
plt.colorbar()
plt.grid(False)
plt.show()

The scale of grey is now from 0 to 1.

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

###**Creation of a validation set**

We further split the train set into a train and validation set. The validation set will be 20% from the original train set, therefore the split will be train/validation of 0.8/0.2.

The actual training set will be divided into two groups:
* the training set
* the validation set

In [ ]:

train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=2020)

In [ ]:
train_id=train_images.index
valid_id=valid_images.index
test_id=test_images.index

In [ ]:
partition = {'train' : [str(i) for i in train_id], 'validation' : [str(i) for i in valid_id]}

labels={}
for i in train_id:
    labels[str(i)]=train_labels[i]
for i in valid_id:
    labels[str(i)]=valid_labels[i]


### We define a class used not to overflow our GPU.

In [ ]:
batch_size=256
epochs=200

In [ ]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=batch_size, dim=input_shape, n_classes=num_classes, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = data['Image'][int(ID)]

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
train_images=np.asarray([i.astype(np.float32) for i in train_images])

valid_images=np.asarray([i.astype(np.float32) for i in valid_images])

test_images=np.asarray([i.astype(np.float32) for i in test_images])

#Generators

training_generator = DataGenerator(partition['train'], labels)
validation_generator = DataGenerator(partition['validation'], labels)


In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)


time0=time.time()
wholemodel_cnn2 = Sequential()
wholemodel_cnn2.add(Rescaling(1./255, input_shape=input_shape))
wholemodel_cnn2.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
wholemodel_cnn2.add(LayerNormalization())
wholemodel_cnn2.add(MaxPooling2D(pool_size=3))
wholemodel_cnn2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
wholemodel_cnn2.add(MaxPooling2D(pool_size=2))
wholemodel_cnn2.add(Dropout(0.1))
wholemodel_cnn2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
wholemodel_cnn2.add(MaxPooling2D(pool_size=2))
wholemodel_cnn2.add(Dropout(0.2))
wholemodel_cnn2.add(Flatten(name='flatten'))
wholemodel_cnn2.add(Dense(num_classes*10, activation='relu'))
wholemodel_cnn2.add(Dense(num_classes*5, activation='relu'))
wholemodel_cnn2.add(Dense(num_classes, activation='softmax'))

wholemodel_cnn2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


# Run the train
wholehistory_cnn2 = wholemodel_cnn2.fit(training_generator,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])

In [ ]:
wholescore_cnn2 = wholemodel_cnn2.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_cnn2[0])
print('Test accuracy:', wholescore_cnn2[1])
time1=time.time()-time0

# Re-training du VGG16

In [ ]:
from keras.applications import VGG16
from keras import models

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False ## Not trainable weights


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(num_classes*10, activation='relu')
dense_layer_2 = layers.Dense(num_classes*5, activation='relu')
prediction_layer = layers.Dense(num_classes, activation='softmax')


wholemodel = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

wholemodel.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

wholehistory_model=wholemodel.fit(training_generator,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])



In [ ]:
valid_set= (valid_images,valid_labels)

In [ ]:
wholescore_cnn = wholemodel_cnn.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_cnn[0])
print('Test accuracy:', wholescore_cnn[1])
time1=time.time()-time0


In [ ]:
wholescore_model = wholemodel.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', wholescore_model[0])
print('Test accuracy:', wholescore_model[1])

In [ ]:
#Loss
print("1st CNN MODEL: ")
history_loss(wholehistory_cnn)
print("2nd CNN MODEL:")
history_loss(wholehistory_cnn2)
print("VGG16 MODEL: ")
history_loss(wholehistory_model)


In [ ]:
# Accuracy
print("1st CNN MODEL: ")
history_accuracy(wholehistory_cnn)
print("2nd CNN MODEL:")
history_accuracy(wholehistory_cnn2)
print("VGG16 MODEL: ")
history_accuracy(wholehistory_model)



In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
CNN = ['CNN 1','CNN 2', 'VGG16']
Test_Accuracy = [wholescore_cnn[1],wholescore_cnn2[1],wholescore_model[1]]
ax.bar(CNN,Test_Accuracy)
plt.show()

In [ ]:
def plot_prediction_images(prediction):
    num_rows = 5
    num_cols = 3
    num_images = num_rows*num_cols
    plt.figure(figsize=(2*2*num_cols, 2*num_rows))
    for i in range (num_images):
        plt.subplot(num_rows, 2*num_cols, 2*i+1)
        plot_image(i, prediction[i], test_labels, test_images)
        plt.subplot(num_rows, 2*num_cols, 2*i+2)
        plot_value_array(i, prediction[i], test_labels)
    plt.tight_layout()
    plt.show()

### 1st CNN predictions

In [ ]:
probability_model = tensorflow.keras.Sequential([wholemodel_cnn,
                                         tensorflow.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)
plot_prediction_images(predictions)


### 2nd CNN predictions

In [ ]:
probability_model2 = tensorflow.keras.Sequential([wholemodel_cnn2,
                                         tensorflow.keras.layers.Softmax()])
predictions2 = probability_model2.predict(test_images)
plot_prediction_images(predictions2)

### VGG16 predictions

In [ ]:
probability_model = tensorflow.keras.Sequential([wholemodel,
                                         tensorflow.keras.layers.Softmax()])
predictions_vgg = probability_model.predict(test_images)
plot_prediction_images(predictions_vgg)

In [ ]:
    img = cv.resize(data, dsize=[100, 100])
    pred = probability_model.predict(img)
    st.write("The three most likely tags are, in decreasing probability order : ", pred)